# Data Preparation

In [1]:
import pandas as pd
import holidays

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel


In [2]:
# Laden der Daten
# Passe den Pfad entsprechend an, falls die Daten in einem spezifischen Ordner liegen
df = pd.read_csv('../data/raw/sickness_table.csv', parse_dates=['date'])

In [3]:
# Unnötige Index-Spalte ignorieren
df = df.drop(columns=['Unnamed: 0'])

In [4]:
# Trenne die Zielvariable vom Feature-Set
X = df.drop(columns=['sby_need'])
y = df['sby_need']

## 1. Datenbereinigung (Teil 1)

In der Datenanalyse wurden folgende Aspekte der Datenqualität überprüft:

- **Fehlende Werte:** Es wurde festgestellt, dass keine fehlenden Werte in den Daten vorliegen. Somit sind keine Imputationsmethoden oder Auffüllstrategien notwendig.
- **Duplikate:** Die Daten wurden auf Duplikate überprüft, und es konnte bestätigt werden, dass keine doppelten Einträge vorhanden sind. Ein Entfernen doppelter Zeilen ist daher nicht erforderlich.
- **Ausreißer:** Während der Analyse wurden mögliche Ausreißer in den numerischen Variablen untersucht. Die vorhandenen Ausreißer scheinen jedoch alle plausibel und nachvollziehbar zu sein (z. B. Spitzenwerte bei Notrufen oder Krankenständen zu bestimmten Zeiten). Daher wurde entschieden, die Ausreißer nicht weiter zu beschneiden oder zu entfernen.

## 2. Feature Engineering

### 2.1. Feature Enrichment

In [7]:
# Custom Transformer für Datum-Features
class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, date_column='date'):
        self.date_column = date_column

    def fit(self, X, y=None):
        # Dynamisch Feiertage für den Bereich des Datum-Features festlegen
        years = range(X[self.date_column].dt.year.min(), X[self.date_column].dt.year.max() + 1)
        self.de_holidays = holidays.Germany(years=years)
        return self
    
    def transform(self, X):
        X = X.copy()
        X['month'] = X[self.date_column].dt.month
        X['day_of_week'] = X[self.date_column].dt.weekday.astype('category')  # 0=Monday, 6=Sunday
        X['quarter'] = X[self.date_column].dt.quarter.astype('category')
        
        # Berechnung der season vor der Typumwandlung in 'category'
        X['season'] = (X['month'] % 12 // 3 + 1).astype('category')  # 1=Winter, 2=Spring, etc.
        
        # Konvertiere 'month' in Kategorie, nachdem season berechnet wurde
        X['month'] = X['month'].astype('category')
        X['holiday'] = X[self.date_column].apply(lambda x: 1 if x in self.de_holidays else 0).astype(bool)
        X['weekend'] = X['day_of_week'].apply(lambda x: 1 if x >= 5 else 0).astype(bool)
        return X

# Custom Transformer für Lag- und Rolling-Features (in Englisch)
class LagRollingFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, lag_features, rolling_features, lags=[7, 30], windows=[7, 30]):
        self.lag_features = lag_features
        self.rolling_features = rolling_features
        self.lags = lags
        self.windows = windows
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for lag in self.lags:
            for feature in self.lag_features:
                X[f'{feature}_lag_{lag}'] = X[feature].shift(lag)
                
        for window in self.windows:
            for feature in self.rolling_features:
                X[f'{feature}_ma_{window}'] = X[feature].rolling(window=window).mean()
                X[f'{feature}_var_{window}'] = X[feature].rolling(window=window).var()
        
        # Entfernen von NaN-Werten, die durch Lag- und Rolling-Features entstehen
        X = X.dropna().reset_index(drop=True)
        return X

# Custom Transformer zum Entfernen der Datetime-Spalte
class DropDateColumn(BaseEstimator, TransformerMixin):
    def __init__(self, date_column='date'):
        self.date_column = date_column
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(columns=[self.date_column])

# Feature-Engineering-Pipeline
feature_engineering_pipeline = Pipeline([
    ('date_features', DateFeatureExtractor(date_column='date')),
    ('lag_rolling_features', LagRollingFeatures(
        lag_features=['calls', 'n_sick', 'dafted'],
        rolling_features=['calls', 'n_sick', 'dafted']
    )),
    ('drop_date', DropDateColumn(date_column='date'))
])

# Pipeline auf die Daten anwenden
df_transformed = feature_engineering_pipeline.fit_transform(X)
df_transformed

,n_sick,calls,n_duty,n_sby,dafted,month,day_of_week,quarter,season,holiday,...,n_sick_ma_7,n_sick_var_7,dafted_ma_7,dafted_var_7,calls_ma_30,calls_var_30,n_sick_ma_30,n_sick_var_30,dafted_ma_30,dafted_var_30
0,54,8400.0,1700,90,0.0,5,6,2,2,True,...,64.000000,29.000000,0.000000,0.000000,6784.2,1.001451e+06,60.700000,49.113793,0.100000,0.300000
1,63,7782.0,1700,90,0.0,5,0,2,2,False,...,62.714286,16.571429,0.000000,0.000000,6759.4,9.305297e+05,60.666667,48.919540,0.100000,0.300000
2,61,7716.0,1700,90,0.0,5,1,2,2,False,...,62.142857,15.809524,0.000000,0.000000,6747.0,9.010570e+05,60.433333,47.012644,0.100000,0.300000
3,58,8340.0,1700,90,0.0,5,2,2,2,False,...,61.571429,18.285714,0.000000,0.000000,6790.2,9.835899e+05,60.000000,43.172414,0.100000,0.300000
4,57,8064.0,1700,90,0.0,5,3,2,2,True,...,60.285714,16.571429,0.000000,0.000000,6817.8,1.031899e+06,59.800000,43.131034,0.100000,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,86,8544.0,1900,90,0.0,5,3,2,2,False,...,78.571429,63.619048,26.714286,4995.571429,9644.2,1.184878e+06,77.500000,39.637931,98.833333,16236.557471
1118,81,8814.0,1900,90,0.0,5,4,2,2,False,...,77.571429,48.619048,26.714286,4995.571429,9581.6,1.166733e+06,77.166667,33.660920,90.833333,15820.005747
1119,76,9846.0,1900,90,56.0,5,5,2,2,False,...,78.142857,43.476190,8.000000,448.000000,9614.4,1.150174e+06,76.966667,32.860920,92.700000,15573.734483
1120,83,9882.0,1900,90,70.0,5,6,2,2,False,...,78.857143,46.809524,18.000000,961.333333,9659.4,1.110149e+06,77.066667,33.788506,95.033333,15289.550575


Durch die oben genannten Schritte wurde die Zeitstruktur der Daten aufgeschlüsselt und wichtige historische Informationen eingebracht. Diese vorbereiteten Features bieten eine starke Grundlage für das Modelltraining, indem sie saisonale Muster und historische Trends berücksichtigen. Mit diesen zusätzlichen Informationen kann das Modell saisonale Zyklen und vergangenheitsbasierte Abhängigkeiten effektiver erfassen, was die Genauigkeit der Vorhersagen verbessern sollte.

## 1. Zeitbasierte Features extrahieren
- Erstellte Features:
    - **Monat:** Gibt den Monat des jeweiligen Datums an (Werte: 1 bis 12).
    - **Wochentag:** Gibt den Wochentag des jeweiligen Datums an (Werte: 0 für Montag bis 6 für Sonntag).
    - **Quartal:** Zeigt an, in welchem Quartal das Datum liegt (Werte: 1 bis 4).
    - **Jahreszeit:** Zeigt die Jahreszeit an, in der das Datum liegt (Werte: 1 für Winter, 2 für Frühling, 3 für Sommer, 4 für Herbst).
- **Begründung:** Zeitbasierte Features sind entscheidend für Zeitreihen, da sie saisonale Muster und Zyklen erfassen. Zum Beispiel können Notrufzahlen und Krankenstände zu bestimmten Jahreszeiten oder Wochentagen höher oder niedriger sein. Durch das Hinzufügen von Features wie Monat, Wochentag, Quartal und Jahreszeit kann das Modell saisonale Schwankungen und wöchentliche Trends besser erkennen.

## 2. Lag-Features erstellen
- Erstellte Features:
    - **Lag-Features:** Werte der Variablen Anzahl_Notrufe, Anzahl_Krankenstand, Ersatzfahrer_aktiviert und Zusätzliche_Fahrer_erforderlich mit Verzögerungen von 7 und 30 Tagen (z. B. Anzahl_Notrufe_Lag_7 und Anzahl_Notrufe_Lag_30).
- **Begründung:** Lag-Features ermöglichen es dem Modell, auf vergangene Informationen zuzugreifen und Trends oder Muster in der Vergangenheit zu berücksichtigen. Für Zeitreihen ist es besonders nützlich zu wissen, wie sich eine Variable in den vorhergehenden Tagen entwickelt hat. Zum Beispiel könnte die Anzahl der Notrufe vor 7 oder 30 Tagen einen Einfluss auf die heutige Anzahl haben. Durch die Einführung von 7-Tage- und 30-Tage-Lags erhält das Modell Informationen über wöchentliche und monatliche Verzögerungen, was bei der Vorhersage saisonaler Muster hilft.

## 3. Gleitende Durchschnitte und Varianzen berechnen
- Erstellte Features:
    - **Gleitende Durchschnitte:** 7-Tage- und 30-Tage-Durchschnitte der Variablen Anzahl_Notrufe, Anzahl_Krankenstand, Ersatzfahrer_aktiviert und Zusätzliche_Fahrer_erforderlich (z. B. Anzahl_Notrufe_MA_7 und Anzahl_Notrufe_MA_30).
    - **Gleitende Varianzen:** 7-Tage- und 30-Tage-Varianzen der gleichen Variablen (z. B. Anzahl_Notrufe_Var_7 und Anzahl_Notrufe_Var_30).
- **Begründung:** Gleitende Durchschnitte (Moving Averages) glätten kurzfristige Schwankungen und geben dem Modell ein klareres Bild von längerfristigen Trends. Die 7-Tage-Durchschnitte erfassen wöchentliche Muster, während die 30-Tage-Durchschnitte längere saisonale Trends widerspiegeln. Gleitende Varianzen (Moving Variances) geben an, wie stark die Werte innerhalb eines bestimmten Zeitfensters schwanken, was Hinweise auf die Stabilität oder Instabilität eines Merkmals geben kann. Wenn beispielsweise die Varianz in den Krankenständen in bestimmten Zeitfenstern hoch ist, könnte dies auf besondere Ereignisse oder Einflüsse hindeuten.

## 4. Entfernen von NaN-Werten
- Durchgeführte Schritte:
    - Entfernen von Zeilen mit NaN-Werten, die durch die Erstellung der Lag- und Moving Average-Features entstanden sind.
- **Begründung:** Lag- und Rolling-Funktionen erzeugen am Anfang der Zeitreihe NaN-Werte, da für die ersten Zeilen keine Daten für die Berechnung der zurückliegenden Tage vorhanden sind. Diese Zeilen wurden entfernt, da sie keine vollständigen Daten enthalten und daher für das Modelltraining ungeeignet wären.

## 5. Feiertag
- **Beschreibung:** In der Spalte Feiertag wird überprüft, ob das jeweilige Datum in die Liste der deutschen Feiertage fällt. Dies wurde mithilfe der holidays-Bibliothek umgesetzt, die für Deutschland und spezifische Jahre Feiertage ausgibt. Für Feiertage erhält das Feature den Wert 1, für normale Tage den Wert 0.
 
- **Begründung:** Feiertage können einen wesentlichen Einfluss auf das Verhalten und die Nachfrage im Rettungsdienst haben. An Feiertagen kann es z. B. zu einer erhöhten Anzahl von Notrufen kommen (z. B. an Silvester) oder die Krankenstände können abweichen. Diese Tage haben oft einzigartige Muster, die ein Modell ohne Berücksichtigung der Feiertage nicht erkennen könnte. Durch das Hinzufügen dieses Features erhält das Modell zusätzliche Informationen, die helfen, besondere Tage zu identifizieren und diese in der Prognose zu berücksichtigen.
## 6. Wochenende
- **Beschreibung:** Die Spalte Wochenende gibt an, ob das Datum auf ein Wochenende fällt (Samstag oder Sonntag). Dabei erhalten Samstage und Sonntage den Wert 1, alle anderen Tage den Wert 0.

- **Begründung:** Wochenenden haben häufig andere Muster im Vergleich zu Werktagen, sowohl in Bezug auf Notrufe als auch auf Krankenstände. Beispielsweise kann die Anzahl der Notrufe am Wochenende variieren, da die Aktivität in der Bevölkerung anders ist als an Werktagen. Auch die Krankenstände könnten am Wochenende anders verlaufen. Dieses Feature hilft dem Modell, die Unterschiede zwischen Wochentagen und Wochenenden zu lernen und die Vorhersage entsprechend zu verbessern.


### 2.2. Scaling und Encoding

In [9]:
# Pipeline für Lineare Regression (nur Scaling und Encoding)
preprocessor_lr = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
            ('cat', OneHotEncoder(drop='first'), make_column_selector(dtype_include='category'))
        ], remainder='passthrough'))  # Alle übrigen Spalten im Originalzustand belassen
])

# Pipeline für Tree-basierte Modelle (nur Encoding, keine Skalierung)
preprocessor_tree = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', 'passthrough', make_column_selector(dtype_include=['int64', 'float64'])),
            ('cat', OneHotEncoder(drop='first'), make_column_selector(dtype_include='category'))
        ], remainder='passthrough'))  # Alle übrigen Spalten im Originalzustand belassen
])

# Anwendung der Pipeline auf die Daten
df_transformed_scaled_encoded = preprocessor_tree.fit_transform(df_transformed)

df_transformed_scaled_encoded

array([[5.400e+01, 8.400e+03, 1.700e+03, ..., 0.000e+00, 1.000e+00,
        1.000e+00],
       [6.300e+01, 7.782e+03, 1.700e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [6.100e+01, 7.716e+03, 1.700e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [7.600e+01, 9.846e+03, 1.900e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [8.300e+01, 9.882e+03, 1.900e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [7.700e+01, 8.790e+03, 1.900e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [ ]:
#### Feature Selection einfügen ###############

## Modellabhängige Datenvorbereitung
Ich habe mich entschieden, unterschiedliche Pipelines für verschiedene Modellgruppen zu definieren, um eine effiziente und modellgerechte Datenvorbereitung zu ermöglichen. Zusätzlich habe ich PCA und Feature Selection integriert, um die Dimensionen zu reduzieren und die relevantesten Merkmale für jedes Modell hervorzuheben.

- Lineare Regression (Baseline-Modell)
    - **Zweck:** Die Lineare Regression dient als Baseline-Modell, um die Modellleistung mit einer einfachen, interpretierten Methode zu vergleichen.
    - **Encoding und Skalierung:** Für die Lineare Regression verwende ich One-Hot-Encoding für die kategorischen Variablen (Monat, Wochentag, Quartal, Jahreszeit), um den linearen Zusammenhang zwischen den einzelnen Kategorien und dem Zielwert explizit zu erfassen. Zudem wird ein StandardScaler auf die numerischen Variablen angewendet, da lineare Modelle empfindlich auf unterschiedliche Skalen reagieren.
    - **PCA:** Zusätzlich habe ich PCA (Principal Component Analysis) in die Pipeline integriert, um die Anzahl der Merkmale zu reduzieren und so die Berechnungen zu beschleunigen. Die PCA ist besonders hilfreich für die Lineare Regression, da sie das Modell auf die wichtigsten Hauptkomponenten fokussiert, ohne die interpretierbare Varianz zu verlieren. Die Anzahl der Hauptkomponenten wurde so gewählt, dass 95 % der Varianz erhalten bleiben, was eine gute Balance zwischen Komplexitätsreduktion und Informationserhalt bietet.

- Random Forest und Gradient Boosting
    - **Zweck:** Diese Modelle sind in der Lage, nichtlineare Zusammenhänge und Interaktionen zwischen den Variablen zu erfassen und robust gegen Ausreißer.
    - **Encoding und Skalierung:** Da baumbasierte Modelle unempfindlich gegenüber unterschiedlichen Skalen sind, lasse ich die numerischen Variablen unverändert (passthrough). Bei den kategorischen Variablen setze ich One-Hot-Encoding ein, um die Interpretierbarkeit der Features zu erhöhen und Redundanzen in den Baumstrukturen zu vermeiden. Alternativ könnte auch ein Label-Encoding für größere Datensätze verwendet werden, wenn Rechenzeit optimiert werden soll.


# Train-Test-Split

In [ ]:
# Setze die Zielvariable
target = 'Ersatzfahrer_aktiviert'

# Definiere die Merkmale (X) und die Zielvariable (y)
X = df.drop(columns=[target])
y = df[target]

# Führe einen zeitlichen Train-Test-Split durch (80 % Training, 20 % Test) mit shuffle=False
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Überprüfen der Aufteilung
print("Training Set Größe:", X_train.shape, y_train.shape)
print("Test Set Größe:", X_test.shape, y_test.shape)

Nach dem Data Preprocessing (Bereinigung und Enrichment) habe ich die Daten in Trainings- und Testsets aufgeteilt. Diese Aufteilung dient dazu, das Modell auf historischen Daten zu trainieren und seine Leistung auf zukünftigen, bislang unbekannten Daten zu bewerten. Dies ist besonders wichtig für die Zeitreihenprognose, da wir sicherstellen möchten, dass das Modell keine „Zukunftsinformationen“ aus dem Testset erhält.

# Begründung
- **Vermeidung von Datenlecks:** Da es sich um zeitbasierte Daten handelt, ist es entscheidend, dass die Testdaten den Modellaufbau nicht beeinflussen, um die zukünftige Vorhersageleistung korrekt abzubilden. Ein zeitlicher Split verhindert, dass Informationen aus der „Zukunft“ in das Modelltraining einfließen.
- **Replizierbarkeit und Robustheit:** Die feste Datenaufteilung gewährleistet, dass die Modellleistung auch bei zukünftigen Daten realistisch ist und nicht auf Muster im Testset abgestimmt wurde.
- **Verallgemeinerungsfähigkeit:** Dieser Ansatz stellt sicher, dass das Modell generalisiert und saisonale Trends sowie kurzfristige Schwankungen auf Basis der Trainingsdaten erkennt.

## Speicherung der prozessierten Daten

In [ ]:
# Sicherstellen, dass der Ordner 'data/processed' existiert
os.makedirs('../data/processed', exist_ok=True)

# Speichern der Trainings- und Testdaten für lineare Modelle
pd.DataFrame(X_train_lr_pca).to_csv('../data/processed/X_train_lr_pca.csv', index=False)
pd.DataFrame(X_test_lr_pca).to_csv('../data/processed/X_test_lr_pca.csv', index=False)

# Speichern der Trainings- und Testdaten für baumbasierte Modelle
pd.DataFrame(X_train_tree_fs).to_csv('../data/processed/X_train_tree_fs.csv', index=False)
pd.DataFrame(X_test_tree_fs).to_csv('../data/processed/X_test_tree_fs.csv', index=False)

# Zielwerte speichern
pd.DataFrame(y_train).to_csv('../data/processed/y_train.csv', index=False)
pd.DataFrame(y_test).to_csv('../data/processed/y_test.csv', index=False)